## 매출에 영향력 높은 후행 지표 찾기

In [294]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
base_path = "C:/Users/dukkubi"
df = pd.read_csv(base_path + "영업팀 매출 데이터.csv")
form = pd.read_csv(base_path + "피터팬의 좋은방구하기 입점문의 접수(응답).csv")
agent = pd.read_csv(base_path + "중개사 데이터.csv")
first_party_db = pd.read_csv(base_path + "first_party_db.csv")

##### 예시 데이터

In [296]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# 랜덤 데이터 생성을 위한 seed 설정
random.seed(42)

# 데이터 크기 설정
num_rows = 1000

# 데이터 생성
regions = ['서울 중구', '서울특별시 강남구', '서울시 도봉구', '부산 서구', '부산광역시 동구', '대구 ', '광주 남구', '대전', '울산', '인천', '제주 서귀포시']
companies = ['피터팬의좋은방구하기 공인중개사', '피터팬의 좋은방구하기 공인중개사', '피터팬의좋은방구하기', '피터팬의좋은방구하기 중개사', '피터팬의좋은방구하기 부동산', '피터팬의좋은방구하기 사무소',
             '직방 공인중개사', '직방 공인중개사', '직방', '직방 중개사', '직방 부동산', '직방 사무소',
             '다방 공인중개사', '다방 공인중개사', '다방', '다방 중개사', '다방 부동산', '다방사무소',
             '한방 공인중개사', '한방 공인중개사', '한방', '한방 중개사', '한방 부동산', '한방 부동산사무소',
             '부동산119 공인중개사', '부동산119 공인중개사', '부동산119', '부동산119 중개사', '부동산119 부동산',
             '리치고 공인중개사', '리치고 공인중개사', '리치고', '리치고 중개사', '리치고 부동산',
             ]
persons = ['김건영', '김건영 ', '김건영(대표자)', ' 김건영', ' 김건영 ',
           '중개사1', '중개사2', '중개사3', '중개사4', '중개사5', '중개사6', '중개사7', '중개사8'
             ]

# 열 데이터 생성
지역 = np.random.choice(regions, num_rows)
상호 = np.random.choice(companies, num_rows)
대표자 = np.random.choice(persons, num_rows)

# 다양한 형식의 매출액 데이터 생성
매출액 = [
    random.choice([
        random.randint(10000, 1000000),  # 숫자
        '',  # 빈 문자열
        f"{random.randint(1000, 1000000):,}",  # 쉼표 포함 숫자
        f"-{random.randint(1000, 100000)}"  # 음수
    ]) for _ in range(num_rows)
]

# 음수 매출액의 경우 '환불'로 설정
def generate_신규연장(value):
    if isinstance(value, str) and value.startswith('-'):
        return '환불'
    return np.random.choice(['신규', '연장', None], p=[0.5, 0.4, 0.1])

신규연장 = [generate_신규연장(value) for value in 매출액]

날짜 = [(datetime.today() - timedelta(days=random.randint(0, 365 * 2))).strftime('%Y-%m-%d') for _ in range(num_rows)]
register_numbers = [f"RN{str(i).zfill(6)}" for i in range(1, num_rows + 1)]

# 데이터프레임 생성
df = pd.DataFrame({
    '지역': 지역,
    '상호': 상호,
    '대표자': 대표자,
    '매출액': 매출액,
    '날짜': 날짜,
    '신규연장': 신규연장
})

# first_party_db 생성
first_party_db = pd.DataFrame({
    '지역': df['지역'],
    '상호': df['상호'],
    '대표자': df['대표자'],
    'register_number': register_numbers
}).drop_duplicates()

### 1. 전처리

#### 1-1. 중개사 식별 key 만들기
영업팀 매출 데이터에 중개사를 식별하는 key가 존재하지 않는다.<br>
따라서, 지역 + 상호 + 대표자 데이터를 공공 데이터의 지역 + 상호 + 대표자와 join 하여 등록번호를 부여했다.<br>
공공 데이터의 경우, vworld 사이트에서 중개사 데이터를 크롤링 후, MySQL에 적재하였다.

**정규식으로 전처리**<br>
영업팀 매출 시트 데이터에서 아래에 해당하는 부분만 제거

상호
- 공인
- 중개사
- 중개
- 사무소
- 부동산
- 법인
- 법인분
- 공백 앞,뒤,가운데
- 주식회사
- 분사무소
- (주)
- 괄호 + 괄호 안에 문자
- .
- 선발행요청
- 소문자는 전부 대문자로 변경


대표자 이름
- 앞뒤공백
- 대괄호 + 괄호 안에 문자
- 소괄호() + 괄호 안에 문자
- (뒤에 문자 제거
 [뒤에 문자 제거


시도
- 광역
- 특별
- 경기도

In [297]:
import re

def preprocess_sangho(text):
    try:
        if isinstance(text, str):
            # 정규식 패턴들
            patterns = [
                r'(공인|중개사|중개|분사무소|사무소|부동산|법인분|법인|주식회사)',
                r'\([^()]*\)',  # 괄호와 괄호 안의 문자 모두 삭제
                r'\s+',
                r'\.',          # 점 삭제
                r'선발행요청'
            ]

            # 각 패턴을 제거하고 대문자로 변환
            for pattern in patterns:
                text = re.sub(pattern, '', text)

            # 대문자로 변환
            text = text.upper()
    except Exception as e:
        pass

    return text


def preprocess_name(text):
    try:
        if isinstance(text, str):
            # 정규식 패턴들
            patterns = [
                r'\s+',  # 공백 제거
                r'\[(.*?)\]',  # 대괄호 안의 문자 제거
                r'\((.*?)\)',  # 소괄호 안의 문자 제거
                r'\(([^()]*)$',  # 소괄호 뒤에 문자 제거
                r'\[([^[\]]*)$'  # 대괄호 뒤에 문자 제거
            ]

            # 각 패턴을 제거하고 대문자로 변환
            for pattern in patterns:
                text = re.sub(pattern, '', text)

    except Exception as e:
        pass

    return text


def preprocess_sido(text):
    try:
        if isinstance(text, str):
            # 정규식 패턴들
            patterns = [
                r'\s+',  # 공백 제거,
                r'광역|특별|경기도|시'
            ]

            # 각 패턴을 제거
            for pattern in patterns:
                text = re.sub(pattern, '', text)

    except Exception as e:
        pass

    return text

In [298]:
# 테스트 문자열
text = "aAg 하남(주)중개사중개 (하남) 사무소 부동산 법인 법인분 주식회사 분사무소 . 선발행요청"

# 전처리된 문자열 출력
print(preprocess_sangho(text))

AAG하남


In [299]:
# 테스트 문자열
text = "김건영[건영킴](건영킴) (11 [22 (33 )"

# 전처리된 문자열 출력
print(preprocess_name(text))

김건영


In [300]:
# 테스트 문자열
text = "서울특별시 강남구 특별시 서울 도봉구 서울시 경기도 수원시 광역시 부산광역시 해운대구 대구광역시 광주광역시 서울특별시 제주도 서귀포시"

# 전처리된 문자열 출력
print(preprocess_sido(text))

서울강남구서울도봉구서울수원부산해운대구대구광주서울제주도서귀포


In [301]:
# 정규식 적용
df['상호'] = df['상호'].apply(lambda x: preprocess_sangho(x))
df['대표자'] = df['대표자'].apply(lambda x: preprocess_name(x))
df['지역'] = df['지역'].apply(lambda x: preprocess_sido(x))

first_party_db['상호'] = first_party_db['상호'].apply(lambda x: preprocess_sangho(x))
first_party_db['대표자'] = first_party_db['대표자'].apply(lambda x: preprocess_name(x))
first_party_db['지역'] = first_party_db['지역'].apply(lambda x: preprocess_sido(x))

In [302]:
# 지역 + 상호 + 대표자 합치기
df['지역 상호 대표자'] = df['지역'] + ' ' + df['상호'] + ' ' + df['대표자']
first_party_db['지역 상호 대표자'] = first_party_db['지역'] + ' ' + first_party_db['상호'] + ' ' + first_party_db['대표자']
print(len(df['지역 상호 대표자'].unique()), len(first_party_db['지역 상호 대표자'].unique()))

427 427


In [303]:
# 데이터 이상 없는지 학인
print(df['지역'].unique(), "\n", df['상호'].unique(), "\n", df['대표자'].unique())

['제주서귀포' '부산동구' '광주남구' '인천' '서울도봉구' '서울강남구' '대구' '부산서구' '대전' '서울중구' '울산'] 
 ['119' '다방' '리치고' '직방' '한방' '피터팬의좋은방구하기'] 
 ['중개사3' '김건영' '중개사2' '중개사7' '중개사4' '중개사6' '중개사5' '중개사8' '중개사1']


In [304]:
# 영업팀 매출 시트와 vworld 공공 데이터를 '지역 + 상호 + 대표자' 기준으로 left join
df['지역 상호 대표자'] = df['지역']  + ' ' + df['상호'] + ' ' + df['대표자']

first_party_db['지역 상호 대표자'] = df['지역']  + ' ' + df['상호'] + ' ' + df['대표자']
first_party_db = first_party_db[['지역 상호 대표자', 'register_number']]

df = df.merge(first_party_db, on='지역 상호 대표자', how='left')
df

,지역,상호,대표자,매출액,날짜,신규연장,지역 상호 대표자,register_number
0,제주서귀포,119,중개사3,"117,739",2024-11-12,신규,제주서귀포 119 중개사3,RN000001
1,부산동구,다방,김건영,266787,2023-09-03,연장,부산동구 다방 김건영,RN000002
2,부산동구,다방,김건영,266787,2023-09-03,연장,부산동구 다방 김건영,RN000031
3,부산동구,다방,김건영,266787,2023-09-03,연장,부산동구 다방 김건영,RN000148
4,부산동구,다방,김건영,266787,2023-09-03,연장,부산동구 다방 김건영,RN000196
...,...,...,...,...,...,...,...,...
4171,대전,리치고,김건영,,2023-08-12,신규,대전 리치고 김건영,RN000298
4172,대전,리치고,김건영,,2023-08-12,신규,대전 리치고 김건영,RN000308
4173,대전,리치고,김건영,,2023-08-12,신규,대전 리치고 김건영,RN000400
4174,대전,리치고,김건영,,2023-08-12,신규,대전 리치고 김건영,RN000511


In [305]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4176 entries, 0 to 4175
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   지역               4176 non-null   object
 1   상호               4176 non-null   object
 2   대표자              4176 non-null   object
 3   매출액              4176 non-null   object
 4   날짜               4176 non-null   object
 5   신규연장             3864 non-null   object
 6   지역 상호 대표자        4176 non-null   object
 7   register_number  4176 non-null   object
dtypes: object(8)
memory usage: 293.6+ KB


#### 1-2. 매출액 전처리

In [306]:
df['매출액']

0       117,739
1        266787
2        266787
3        266787
4        266787
         ...   
4171           
4172           
4173           
4174           
4175           
Name: 매출액, Length: 4176, dtype: object

In [307]:
# 매출액 전처리 함수
def preprocess_sales(sales):
    try:
        # 빈 문자열은 0으로 처리
        if sales == '':
            return 0
        # 문자열일 경우 쉼표 제거 후 숫자로 변환
        if isinstance(sales, str):
            return int(sales.replace(',', ''))
        # 이미 숫자인 경우 그대로 반환
        return int(sales)
    except ValueError:
        # 변환 실패 시 NaN 반환
        return np.nan

# 매출액 열 전처리
df['매출액'] = df['매출액'].apply(preprocess_sales)

# 결과 출력
print(df['매출액'])

0       117739
1       266787
2       266787
3       266787
4       266787
         ...  
4171         0
4172         0
4173         0
4174         0
4175         0
Name: 매출액, Length: 4176, dtype: int64


#### 1-3. '신규연장' 열에 데이터가 없는 경우 '신규' 인지, '연장'인지 
- '신규연장' 열은 최초구매 데이터인지, 재구매 데이터인지, 환불 데이터인지를 구분해주는 Columns이다.

In [308]:
df['신규연장'].value_counts(dropna=False)

신규     1548
연장     1206
환불     1110
NaN     312
Name: 신규연장, dtype: int64

In [309]:
nan_agent_list = df.loc[df['신규연장'].isna(), '지역 상호 대표자'].unique().tolist()
nan_agent_first_purchase_date = df.loc[df['지역 상호 대표자'].isin(nan_agent_list)].groupby(['지역 상호 대표자'])['날짜'].min().reset_index().rename(columns={'날짜':'최초구매날짜'})
nan_agent_first_purchase_date.head()

,지역 상호 대표자,최초구매날짜
0,광주남구 119 중개사8,2024-11-05
1,광주남구 직방 김건영,2023-02-21
2,광주남구 직방 중개사3,2024-05-05
3,광주남구 피터팬의좋은방구하기 중개사6,2023-09-17
4,광주남구 한방 김건영,2023-02-05


In [310]:
df_merge_first_purchase = df.merge(nan_agent_first_purchase_date, how='left', on='지역 상호 대표자')
df_merge_first_purchase.loc[(~df_merge_first_purchase['최초구매날짜'].isna()) & df_merge_first_purchase['신규연장'].isna()]

,지역,상호,대표자,매출액,날짜,신규연장,지역 상호 대표자,register_number,최초구매날짜
35,대구,한방,중개사6,361663,2024-02-04,None,대구 한방 중개사6,RN000012,2024-02-04
36,대구,한방,중개사6,361663,2024-02-04,None,대구 한방 중개사6,RN000386,2024-02-04
85,서울중구,직방,중개사3,712729,2023-08-03,None,서울중구 직방 중개사3,RN000021,2023-08-03
86,서울중구,직방,중개사3,712729,2023-08-03,None,서울중구 직방 중개사3,RN000261,2023-08-03
87,서울중구,직방,중개사3,712729,2023-08-03,None,서울중구 직방 중개사3,RN000393,2023-08-03
...,...,...,...,...,...,...,...,...,...
4159,광주남구,직방,김건영,395548,2024-09-07,None,광주남구 직방 김건영,RN000329,2023-02-21
4160,광주남구,직방,김건영,395548,2024-09-07,None,광주남구 직방 김건영,RN000382,2023-02-21
4161,광주남구,직방,김건영,395548,2024-09-07,None,광주남구 직방 김건영,RN000898,2023-02-21
4162,광주남구,직방,김건영,395548,2024-09-07,None,광주남구 직방 김건영,RN000933,2023-02-21


In [311]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4176 entries, 0 to 4175
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   지역               4176 non-null   object
 1   상호               4176 non-null   object
 2   대표자              4176 non-null   object
 3   매출액              4176 non-null   int64 
 4   날짜               4176 non-null   object
 5   신규연장             3864 non-null   object
 6   지역 상호 대표자        4176 non-null   object
 7   register_number  4176 non-null   object
dtypes: int64(1), object(7)
memory usage: 293.6+ KB


In [312]:
# 조건에 맞게 '신규연장' 컬럼 생성
df_merge_first_purchase['신규연장'] = df_merge_first_purchase.apply(
    lambda row: '신규' if pd.to_datetime(row['날짜']) == pd.to_datetime(row['최초구매날짜']) 
    else ('연장' if pd.to_datetime(row['날짜']) > pd.to_datetime(row['최초구매날짜']) else row['신규연장']),
    axis=1
)

In [313]:
df_merge_first_purchase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4176 entries, 0 to 4175
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   지역               4176 non-null   object
 1   상호               4176 non-null   object
 2   대표자              4176 non-null   object
 3   매출액              4176 non-null   int64 
 4   날짜               4176 non-null   object
 5   신규연장             4176 non-null   object
 6   지역 상호 대표자        4176 non-null   object
 7   register_number  4176 non-null   object
 8   최초구매날짜           1543 non-null   object
dtypes: int64(1), object(8)
memory usage: 326.2+ KB


In [314]:
df = df_merge_first_purchase.drop('최초구매날짜', axis=1)

### 2. 매출 분석

#### 2-1. 신규가입 중개사 vs 재계약(연장) 중개사 비교

In [315]:
# 신규 가입 중개사 수와 재개약(연장) 중개사 수 비교 
print(df.groupby('신규연장', dropna=False).size().reset_index(name='count'))

# 신규 가입 중개사의 매출과 연장 중개사의 매출 비교
print(df.groupby('신규연장', dropna=False)['매출액'].sum().reset_index())

  신규연장  count
0   신규   1359
1   연장   2117
2   환불    700
  신규연장        매출액
0   신규  434371052
1   연장  573033026
2   환불  -35336929


연장 중개사 수, 매출 비율이 더 높은 점을 확인했다.<br>
평균적으로 중개사가 몇 개월 단위로, 몇 회 연장을 하는지도 확인을 해야한다.<br>
<br>
ex) 1개월 당 꾸준히 100명의 신규 중개사 유입
- 평균 연장 횟수: 3회
- 1회 연장 당 평균적으로 걸리는 기간: 3개월<br>

위 조건에서 작년 이번 달 처음 100명의 신규 중개사가 유입되었다고 가정하면 다음과 같은 결과가 나온다.

- 1개월 뒤 : 신규 중개사 100명, 연장 중개사 0명
- 2개월 뒤 : 신규 중개사 100명, 연장 중개사 0명
- 3개월 뒤 : 신규 중개사 100명, 연장 중개사 100명
- 4개월 뒤 : 신규 중개사 100명, 연장 중개사 100명
- 5개월 뒤 : 신규 중개사 100명, 연장 중개사 100명
- 6개월 뒤 : 신규 중개사 100명, 연장 중개사 200명
- 7개월 뒤 : 신규 중개사 100명, 연장 중개사 200명
- 8개월 뒤 : 신규 중개사 100명, 연장 중개사 200명
- 9개월 뒤 : 신규 중개사 100명, 연장 중개사 300명
- 10개월 뒤 : 신규 중개사 100명, 연장 중개사 300명
- 11개월 뒤 : 신규 중개사 100명, 연장 중개사 300명
- 12개월 뒤 : 신규 중개사 100명, 연장 중개사 300명

#### 2-2. 평균 연장 횟수 구하기
**계산 조건**
- 일반적으로 중개사는 3개월 단위로 연장을 하기 때문에 같은 달에 안에서는 여러번 연장을 했어도 1회로 간주(이 경우 재구매(연장)가 아니라 여러번 끊어서 구매했을 확률이 높음)
- 최초구매일 기준으로 12개월치 데이터만 가지고 계산<br>
-> 단순히 최근 2년치 데이터를 전부 가지고 평균을 계산하면, 1개월 전에 신규로 등록한 중개사의 경우 당연히 연장 횟수가 최대 1회일 수밖에 없다.<br>
따라서, 12개월치의 데이터만 추출하여 평균 연장 횟수를 계산했다.<br>
ex) 11개월 전 ~ 현재까지 신규로 등록한 중개사 데이터 제외, 그 이전에 신규로 등록한 중개사는 최대 12개월치 데이터만 가지고 계산

In [316]:
from datetime import date
from dateutil.relativedelta import relativedelta

## 평균 연장 횟수

# 중개사별 첫 구매 날짜
first_purchase_date = df.loc[df['신규연장'] == '신규', ['날짜', 'register_number']].rename(columns={'날짜': '신규가입날짜'})

# 월 단위로 변경
first_purchase_date['신규가입날짜'] =  pd.to_datetime(first_purchase_date['신규가입날짜']).dt.strftime('%Y-%m')

# 기존 데이터에 첫 구매 날짜 join
df = df.merge(first_purchase_date, on='register_number', how='left')

# 연장 날짜 - 첫 구매 날짜 계산
df['연장경과개월'] = (
    (pd.to_datetime(df['날짜']).dt.year - pd.to_datetime(df['신규가입날짜'], format='%Y-%m').dt.year) * 12 +
    (pd.to_datetime(df['날짜']).dt.month - pd.to_datetime(df['신규가입날짜'], format='%Y-%m').dt.month)
)

# 첫 구매일로부터 1개월 단위로 연장한 횟수
df_cohort = df.groupby(['신규가입날짜', '연장경과개월'])['register_number'].nunique().reset_index()

# 11개월 경과까지의 데이터만 선택. 12개월치 데이터가 없는 중개사는 선택 X
year_month_of_12_ago = (date.today() - relativedelta(months=12)).strftime('%Y-%m')
condition1 = (df_cohort['연장경과개월'] < 12)
condition2 = (df_cohort['신규가입날짜'] <= year_month_of_12_ago)
condition3 = (df_cohort['연장경과개월'] != 0)
df_cohort = df_cohort.loc[condition1 & condition2 & condition3]

# 평균 연장 횟수
년월기준신규중개사수 = df_cohort.loc[df_cohort['연장경과개월'] == 1].groupby('신규가입날짜')['register_number'].sum().reset_index()['register_number'].sum()
년월기준연장중개사수 = df_cohort.loc[df_cohort['연장경과개월'] != 1].groupby('신규가입날짜')['register_number'].sum().reset_index()['register_number'].sum()
print("평균연장횟수:", 년월기준연장중개사수/년월기준신규중개사수)

평균연장횟수: 12.54


#### 2-3. 1회 연장 당 평균적으로 걸리는 기간 
- 월 단위로 계산. 같은 달에 연장을 한 경우는 제외 하고 계산(제외하지 않은 경우도 계산해보았음)
- SQL의 윈도우 함수와 같은 방식으로 적용
SELECT register_number, TIMESTAMPDIFF(MONTH, 년월) OVER (PARTITION BY register_number
					   ORDER BY 년월 ACS
					   ROWS PRECENDING) AS month_diff<br>
FROM df

In [317]:
import pandas as pd

# '년월' 컬럼 생성
df['년월'] = pd.to_datetime(df['날짜']).dt.strftime('%Y-%m')

# '년월' 컬럼이 이미 datetime 형식이 아니면 datetime 형식으로 변환
df['년월'] = pd.to_datetime(df['년월'], errors='coerce')

# 'register_number'와 '년월' 기준으로 정렬
df = df.sort_values(by=['register_number', '년월'])

# 두 날짜 간의 차이를 월 단위로 계산하는 함수
def month_diff(start_date, timedelta):
    if pd.notna(start_date) and pd.notna(timedelta):
        end_date = start_date + timedelta  # start_date에 timedelta를 더하여 end_date를 계산
        return (end_date.year - start_date.year) * 12 + end_date.month - start_date.month
    return None


# # 각 'register_number'별로 그룹화하고, '년월' 간 차이를 계산
df['month_diff'] = df.groupby('register_number')['년월'].apply(
    lambda x: x.diff().apply(lambda y: month_diff(x.iloc[0], y) if pd.notna(y) else None)
)

# 결과 출력
print(df[['register_number', '년월', 'month_diff']])

     register_number         년월 month_diff
0           RN000001 2024-11-01       None
7802        RN000002 2023-01-01        NaN
7803        RN000002 2023-01-01        0.0
7804        RN000002 2023-01-01        0.0
7805        RN000002 2023-01-01        0.0
...              ...        ...        ...
3191        RN001000 2024-09-01        0.0
2473        RN001000 2024-10-01        0.0
2474        RN001000 2024-10-01        0.0
2475        RN001000 2024-10-01        0.0
2476        RN001000 2024-10-01        0.0

[8223 rows x 3 columns]


In [318]:
# 1회 연장당 평균적으로 걸리는 기간 계산 결과(같은 달에 연장을 한 경우 포함)
print(df['month_diff'].mean())

1.5236051502145922


In [319]:
# 같은 달에 연장을 한 경우 제외
print(df.loc[df['month_diff'] != 0, 'month_diff'].mean())

4.25725338491296


#### 2-4. 신규 가입 후, 3개월 안에 이탈하는 이탈률 구하기

In [320]:
# 먼저 '연장경과개월'이 3 미만인 그룹과 3 이상인 그룹을 나눕니다.
grouped = df.groupby('register_number')

# 3 미만의 연장경과개월만 있는 그룹을 선택하는 조건
group_3_less = grouped.filter(lambda x: (x['연장경과개월'] < 3).all())

# 3 이상도 포함된 그룹을 선택하는 조건
group_3_or_more = grouped.filter(lambda x: (x['연장경과개월'] >= 3).any())

# 결과 확인
print("3 미만의 연장경과개월만 있는 그룹:")
print(len(group_3_less), "명")

print("\n3 이상도 포함된 그룹:")
print(len(group_3_or_more), "명")


3 미만의 연장경과개월만 있는 그룹:
323 명

3 이상도 포함된 그룹:
7506 명


In [321]:
# 신규 가입 후, 3개월 안에 이탈하는 이탈률 계산 결과
print(round(len(group_3_less)/len(group_3_or_more) * 100,3), "%")

4.303 %


#### 2-5. 3개월 이내에 이탈한 인원 제외한 평균 연장 횟수

In [323]:
# 연장 날짜 - 첫 구매 날짜 계산
group_3_or_more['연장경과개월'] = (
    (pd.to_datetime(group_3_or_more['날짜']).dt.year - pd.to_datetime(group_3_or_more['신규가입날짜'], format='%Y-%m').dt.year) * 12 +
    (pd.to_datetime(group_3_or_more['날짜']).dt.month - pd.to_datetime(group_3_or_more['신규가입날짜'], format='%Y-%m').dt.month)
)

# 첫 구매일로부터 1개월 단위로 연장한 횟수
group_3_or_more_cohort = group_3_or_more.groupby(['신규가입날짜', '연장경과개월'])['register_number'].nunique().reset_index()

# 11개월 경과까지의 데이터만 선택. 12개월치 데이터가 없는 중개사는 선택 X
year_month_of_12_ago = (date.today() - relativedelta(months=12)).strftime('%Y-%m')
condition1 = (group_3_or_more_cohort['연장경과개월'] < 12)
condition2 = (group_3_or_more_cohort['신규가입날짜'] <= year_month_of_12_ago)
condition3 = (group_3_or_more_cohort['연장경과개월'] != 0)
group_3_or_more_cohort = group_3_or_more_cohort.loc[condition1 & condition2 & condition3]

# 평균 연장 횟수
년월기준신규중개사수 = group_3_or_more_cohort.loc[group_3_or_more_cohort['연장경과개월'] == 1].groupby('신규가입날짜')['register_number'].sum().reset_index()['register_number'].sum()
년월기준연장중개사수 = group_3_or_more_cohort.loc[group_3_or_more_cohort['연장경과개월'] != 1].groupby('신규가입날짜')['register_number'].sum().reset_index()['register_number'].sum()
print("평균연장횟수:",년월기준연장중개사수/년월기준신규중개사수)

평균연장횟수: 12.979166666666666


### 3. 1차 후행 지표 선정

1회 연장 당 평균적으로 걸리는 기간은 업계 특성상 쉽게 바뀔 수 없다.<br>
억지로 기간을 줄이려고 하면 역효과가 나타날 수 있다.<br>
또한, 평균 연장 횟수가 같이 늘어나지 않는 이상 연장 기간이 줄어드는건 수치상 큰 의미는 없다.<br>
따라서, 매출을 늘리기 위해 개선이 필요한 지표는 다음과 같다.

**개선이 필요한 지표**
- 신규 중개사 수
- 평균 연장 횟수
- ARPPU

이를 식으로 대략적으로 나타내면 다음과 같다.

$ 매출 = (신규 중개사 수 + (신규 중개사 수 \times 평균 연장 횟수)) * ARPPU $

신규 중개사 수는 단위가 크지만, 평균 연장 횟수는 단위가 작으므로 1회만 늘려도 큰 효과를 볼 수 있다.<br>
ex) 1개월당 꾸준히 유입되는 신규 중개사가 50명이 넘는 상황에서는 신규 중개사 10명 늘리는 것보다 평균 연장 횟수 1회 늘리는 게 더 큰 효과를 본다.<br>

ARPPU는 영업팀, 경영전략팀에서 전담해서 맡고 있기 때문에 ARPPU에 악영향을 주지 않는 방향으로 나머지 2지표에 집중하기로 했다.

- 평균 연장 횟수는 서비스 팀, 마케팅 팀에서 서포트 가능.
- 신규 중개사 수는 마케팅 팀에서 서포트 가능.

평균 연장 횟수에 기여하는 지표가 무엇인지 조사해보았다.

#### 3-1. 3개월 이내 이탈하는 중개사의 이탈률
- 3개월 이내 이탈하는 중개사의 비율을 00%로 꽤 높은 편이다.
- 반면 한 번 연장을 한 중개사의 경우 평균적으로 00번 연장을 했다.
- 중개사들이 3개월 이내에 이탈하지 않게 만들면, 평균 연장 횟수를 많이 높일 수 있다.
- 3개월 이내에 이탈한 중개사 그룹과 이탈하지 않은 그룹의 중개사들의 차이점을 알아보기로 했다.

#### 3-2. 3개월 이내에 이탈하는 중개사 vs 이탈하지 않는 중개사의 평균적으로 받는 문의 횟수

#### 3-3. 두 그룹의 최초 구매시 평균 단가

지금까지는 평균 연장 횟수를 전체 중개사 대상으로 조사했지만, 개별 조사도 필요하다.<br>
평균 연장 횟수의 분산을 통해 상위 20% 그룹, 중간 그룹, 하위 20% 그룹으로 나누었다.

각 그룹 대상으로 다시 지표를 살펴보았다.

#### 유입 채널에 따른 유입수, 전환율 차이